# Rally Stage Report for Dakar Rally

Generate a rally report for a single stage on the Dakar Rally, showing progression across splits.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import Dakar_Rally_2020 as dr

In [175]:
from Dakar_Rally_2020 import get_timing_data_long_timeInSplit, \
                                get_long_annotated_timing_data, TIME, RANK, \
                                rebaseWaypointTimes, pivotRebasedSplits, \
                                get_driver_data, cleanDriverSplitReportBaseDataframe, \
                                get_timing_data, insertColumn, moveColumn, \
                                get_ranking_data, getCurrPrevOverallRank

from dakar_utils import moreStyleDriverSplitReportBaseDataframe, \
                        sparkline2, sparklineStep, getTablePNG
                                
import sqlite3
from sqlite_utils import Database
import pandas as pd

In [312]:
dbname = 'dakar_2020.db'

conn = sqlite3.connect(dbname)
db = Database(conn)

In [313]:
q="SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(q, conn)

,name
0,teams
1,crew
2,vehicles
3,stagestats
4,ranking
5,waypoints


In [91]:
STAGE = 1
VTYPE = 'car'


In [240]:
REBASER = 310 #Alonso - 310
CHART_POSITIONS = 15 #None for all


MAXMISSING = 10

In [480]:
timing_data_long = get_long_annotated_timing_data(STAGE, VTYPE)[TIME]
timing_data_long_insplit = get_timing_data_long_timeInSplit(STAGE, VTYPE)
timing_data_long_min = rebaseWaypointTimes( timing_data_long , REBASER, 'TimeInS')

rb2c = pivotRebasedSplits(rebaseWaypointTimes(timing_data_long_insplit,REBASER))
rb2c = cleanDriverSplitReportBaseDataframe(rb2c, STAGE)
top10 = get_driver_data(STAGE, CHART_POSITIONS)

#rb2cTop10 = rb2c[rb2c.index.isin(top10['Bib'])]
rb2cTop10 = pd.merge(top10, rb2c, how='left', left_index=True, right_index=True)
rb2cTop10.head(2)

,Pos,Road Position,Crew,Brand,D01_km47,D02_km86,D03_km105,D04_km158,D05_km208,D06_km254,D07_km294,D08_ass
Bib,,,,,,,,,,,,
319,1.0,19.0,V. ZALA S. JURGELENAS AGRORODEO,MINI,61.0,-42.0,NaN,165.0,399.0,NaN,335.0,28.0
302,2.0,3.0,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,109.0,38.0,NaN,4783.0,158.0,11.0,293.0,18.0


In [481]:
#Need processing on the below - also change column order
newColOrder = rb2cTop10.columns[1:].tolist()+[rb2cTop10.columns[0]]
rb2cTop10=rb2cTop10[newColOrder]
rb2cTop10 = pd.merge(rb2cTop10, -timing_data_long_min.reset_index().pivot('Bib','Waypoint','rebased'),
                     how='left', left_index=True,right_index=True)


#Cast s to timedelta

#for c in [c for c in rb2cTop10.columns if c.startswith('0')]:
#    rb2cTop10[c]=rb2cTop10[c].apply(lambda x: pd.to_timedelta('{}00:00:{}'.format('-' if x<0 else '', '0' if pd.isnull(x) else abs(x))))

#Rename last column
rb2cTop10.rename(columns={rb2cTop10.columns[-1]:'Stage Overall'}, inplace=True)

rb2cTop10.dropna(how='all',axis=1,inplace=True)

#Drop very gappy waypoint cols
rb2cTop10.dropna(thresh=MAXMISSING,axis=1,inplace=True)

rb2cTop10.head(3)

,Road Position,Crew,Brand,D01_km47,D02_km86,D04_km158,D05_km208,D06_km254,D07_km294,D08_ass,Pos,01_km47,02_km86,04_km158,05_km208,06_km254,07_km294,Stage Overall
Bib,,,,,,,,,,,,,,,,,,
319,19.0,V. ZALA S. JURGELENAS AGRORODEO,MINI,61.0,-42.0,165.0,399.0,NaN,335.0,28.0,1.0,61.0,19.0,165.0,564.0,564.0,899.0,927.0
302,3.0,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,109.0,38.0,4783.0,158.0,11.0,293.0,18.0,2.0,109.0,147.0,313.0,471.0,482.0,775.0,793.0
305,6.0,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,156.0,32.0,4145.0,519.0,50.0,382.0,79.0,3.0,156.0,188.0,-273.0,246.0,296.0,678.0,757.0


In [482]:
from IPython.display import HTML

In [483]:
s2 = moreStyleDriverSplitReportBaseDataframe(rb2cTop10, STAGE)
display(HTML(s2))

,Road Position,Crew,Brand,D01_km47,D02_km86,D04_km158,D05_km208,D06_km254,D07_km294,D08_ass,Pos,01_km47,02_km86,04_km158,05_km208,06_km254,07_km294,Stage Overall
Bib,,,,,,,,,,,,,,,,,,
319,19,V. ZALA S. JURGELENAS AGRORODEO,MINI,61,-42,165,399,0,335,28,1,61,19,165,564,564,899,927
302,3,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,109,38,4783,158,11,293,18,2,109,147,313,471,482,775,793
305,6,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,156,32,4145,519,50,382,79,3,156,188,-273,246,296,678,757
300,1,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA,199,61,4663,464,-378,126,-1,4,199,260,383,847,469,595,594
307,8,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA,78,-45,4604,401,-203,263,16,5,78,33,60,461,258,521,537
311,12,O. TERRANOVA B. GRAUE X-RAID MINI JCW TEAM,MINI,159,-77,187,42,73,303,-113,6,159,82,187,229,302,605,492
315,15,M. SERRADORI F. LURQUIN SRT RACING,CENTURY,144,16,77,261,-55,225,-116,7,144,160,77,338,283,508,392
317,17,V. VASILYEV V. YEVTYEKHOV X-RAID G-ENERGY,MINI,-64,-164,-199,456,-25,-32,-78,8,-64,-228,-199,257,232,200,122
309,10,Y. AL RAJHI K. ZHILTSOV OVERDRIVE TOYOTA,TOYOTA,-448,56,-198,229,28,93,-51,9,-448,-392,-198,31,59,152,101


In [484]:
rb2cTop10['test']= rb2cTop10[[c for c in rb2cTop10.columns if c.startswith(('0','1','Stage Overall'))]].values.tolist()
#Swap the sign of the values
rb2cTop10['test'] = rb2cTop10['test'].apply(lambda x: [-y for y in x])
rb2cTop10.head()

,Road Position,Crew,Brand,D01_km47,D02_km86,D04_km158,D05_km208,D06_km254,D07_km294,D08_ass,Pos,01_km47,02_km86,04_km158,05_km208,06_km254,07_km294,Stage Overall,test
Bib,,,,,,,,,,,,,,,,,,,
319,19.0,V. ZALA S. JURGELENAS AGRORODEO,MINI,61.0,-42.0,165.0,399.0,NaN,335.0,28.0,1.0,61.0,19.0,165.0,564.0,564.0,899.0,927.0,"[-61.0, -19.0, -165.0, -564.0, -564.0, -899.0,..."
302,3.0,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,109.0,38.0,4783.0,158.0,11.0,293.0,18.0,2.0,109.0,147.0,313.0,471.0,482.0,775.0,793.0,"[-109.0, -147.00000000000045, -313.0, -471.0, ..."
305,6.0,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,156.0,32.0,4145.0,519.0,50.0,382.0,79.0,3.0,156.0,188.0,-273.0,246.0,296.0,678.0,757.0,"[-156.0, -188.00000000000045, 273.0, -246.0, -..."
300,1.0,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA,199.0,61.0,4663.0,464.0,-378.0,126.0,-1.0,4.0,199.0,260.0,383.0,847.0,469.0,595.0,594.0,"[-198.99999999999977, -260.00000000000045, -38..."
307,8.0,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA,78.0,-45.0,4604.0,401.0,-203.0,263.0,16.0,5.0,78.0,33.0,60.0,461.0,258.0,521.0,537.0,"[-78.0, -33.0, -60.0, -461.0, -258.0, -521.0, ..."


In [485]:
brandcol = rb2cTop10.columns.get_loc("Brand")
rb2cTop10.insert(brandcol+1, 'Stage Gap', rb2cTop10['test'].apply(sparkline2,typ='bar'))
rb2cTop10.drop('test', axis=1, inplace=True)

In [486]:
display(HTML(rb2cTop10.head(3).style.render()))

,Road Position,Crew,Brand,Stage Gap,D01_km47,D02_km86,D04_km158,D05_km208,D06_km254,D07_km294,D08_ass,Pos,01_km47,02_km86,04_km158,05_km208,06_km254,07_km294,Stage Overall
Bib,,,,,,,,,,,,,,,,,,,
319,19,V. ZALA S. JURGELENAS AGRORODEO,MINI,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJAAAAAkCAYAAABmHbPbAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1%2B/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4xLjEsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy8QZhcZAAAA20lEQVR4nO3ZsQ2CQACGUc7YOIIbsAVbMQRbsQUbOIIllhYQE/1BBN%2BriIpQfLk7uDKO41jBh05b3wD7JiAiAiJyXvXfS5l8dGvb6l7X879vmudx3793LedOXIahunbd9IsFl71l1UX0TEAvLXUryXX/7dzQuiOQB7zDswYiIiAi605hWzF1fo0RiIiAiAiIiICICIiIgIgIiIiAiAiIiICIHHMrI5Fsg/zhFooRiIiAiAiIiICICIiIgIgIiIiAiHiR%2BCt2%2BhJSQEewYXymMCICIiIgIgIiIiAiDyyvND8trSUsAAAAAElFTkSuQmCC"" />",61,-42,165,399,nan,335,28,1,61,19,165,564,564,899,927
302,3,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJAAAAAkCAYAAABmHbPbAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1%2B/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4xLjEsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy8QZhcZAAAA2klEQVR4nO3bwQ2CQBRFUTBuLMEO6IKuKIKu6IIOLMElLjWBmOhzUPCcFVFGWdwMHxPraZqmCt50%2BPYFsG0CIiIgIsein17Xs5cuXVddm2b5/La9Hw/Da99l7cxpHKtz38/f%2BODYWxcdohcCeurxUqxdZ23ILYyIgIiUnYH8xLR7ZQNKiG8T3MKICIiIgIgIiIiAiPzuU1jCE9xq7EBEBEREQEQERGSfQ3QiGcD/cHi3AxEREBEBEREQEQERERARAREREBEBESn7x0J2zw5EREBEBEREQEQEROQGoS44OhRenggAAAAASUVORK5CYII%3D"" />",109,38,4783,158,11,293,18,2,109,147,313,471,482,775,793
305,6,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJAAAAAkCAYAAABmHbPbAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1%2B/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4xLjEsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy8QZhcZAAABCklEQVR4nO3aMQ6DIABGYWi6cASmrt7CW7k2Xb2Vt%2BjaiSM40tXGpgn%2BImDft4qR1BeiWBtjjAbY6FJ6AmgbAUFCQJAQECQEBMm19ARysA%2BbND7eeRHdihUIEgKChIAgISBI7NGfMkIIZp7nrNeYXlPS%2BP7WZ5pJWc45473Peo28Adn121AYBjN33ffx/eJGTmkRfJyrUK5b2bnu%2BTR%2BHNcHdrzlhwf003IqyrmKUnNu8bcyPANBRECQ5N2J5q9Gp1fvpwzia0K9AZVCuEkIaE9/GB8P0ZAQECQEBAkBQUJAkBAQJAQECQFBwkZiLRrdhGQFgoQV6AwKrl6sQJAQECQEBAkBQUJAkLwBwT9GN6y21CsAAAAASUVORK5CYII%3D"" />",156,32,4145,519,50,382,79,3,156,188,-273,246,296,678,757


In [487]:
# get gap to leader at each split
tmp = get_timing_data(STAGE,vtype=VTYPE, timerank='gap', kind='full')[TIME].set_index('Bib')
tmp.dropna(thresh=MAXMISSING,axis=1,inplace=True)
tmp.head(3)

,Pos,Refuel,Crew,Brand,00_dss,01_km47,02_km86,04_km158,05_km208,06_km254,...,km208_pos_05,km208_raw_05,km254_gain_06,km254_pos_06,km254_raw_06,km294_gain_07,km294_pos_07,km294_raw_07,ass_gain_08,ass_raw_08
Bib,,,,,,,,,,,,,,,,,,,,,
319,1.0,False,V. ZALA S. JURGELENAS AGRORODEO,MINI,10:00:00,00:02:18,00:04:01,00:03:38,00:04:43,00:01:11,...,3.0,00:04:43,▲,2.0,00:01:11,▲,1.0,00:00:00,=,00:00:00
302,2.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,09:26:00,00:01:30,00:01:53,00:01:10,00:06:16,00:02:33,...,4.0,00:06:16,▲,3.0,00:02:33,▲,2.0,00:02:04,=,00:02:14
305,3.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,09:35:00,00:00:43,00:01:12,00:10:56,00:10:01,00:05:39,...,9.0,00:10:01,▲,6.0,00:05:39,▲,3.0,00:03:41,=,00:02:50


In [488]:
cols = [c for c in tmp.columns if c.startswith(('0','1'))]
tmp[cols]  = tmp[cols].apply(lambda x: x.dt.total_seconds())
tmp['test']= tmp[[c for c in tmp.columns if (c.startswith(('0','1')) and 'dss' not in c)]].values.tolist()
tmp['test2']= tmp[[c for c in tmp.columns if '_pos' in c]+['Pos']].values.tolist()
#Want better rank higher up
tmp['test2'] = tmp['test2'].apply(lambda x: [-y for y in x])
tmp.head(3)


,Pos,Refuel,Crew,Brand,00_dss,01_km47,02_km86,04_km158,05_km208,06_km254,...,km254_gain_06,km254_pos_06,km254_raw_06,km294_gain_07,km294_pos_07,km294_raw_07,ass_gain_08,ass_raw_08,test,test2
Bib,,,,,,,,,,,,,,,,,,,,,
319,1.0,False,V. ZALA S. JURGELENAS AGRORODEO,MINI,36000.0,138.0,241.0,218.0,283.0,71.0,...,▲,2.0,00:01:11,▲,1.0,00:00:00,=,00:00:00,"[138.0, 241.00000000000003, 218.0, 283.0, 71.0...","[-19.0, -11.0, -10.0, -5.0, -3.0, -2.0, -1.0, ..."
302,2.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,33960.0,90.0,113.0,70.0,376.0,153.0,...,▲,3.0,00:02:33,▲,2.0,00:02:04,=,00:02:14,"[90.0, 113.0, 70.0, 376.0, 153.0, 124.00000000...","[-3.0, -7.0, -6.0, -2.0, -4.0, -3.0, -2.0, -2.0]"
305,3.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,34500.0,43.0,72.0,656.0,601.0,339.0,...,▲,6.0,00:05:39,▲,3.0,00:03:41,=,00:02:50,"[43.0, 72.0, 656.0, 601.0, 339.0, 221.0, 170.0]","[-6.0, -4.0, -2.0, -15.0, -9.0, -6.0, -3.0, -3.0]"


In [489]:
rb3c = get_timing_data(STAGE,vtype=VTYPE, timerank='gap', kind='full')[TIME].set_index('Bib')
rb3c.dropna(thresh=MAXMISSING,axis=1,inplace=True)
rb3c.head(3)

,Pos,Refuel,Crew,Brand,00_dss,01_km47,02_km86,04_km158,05_km208,06_km254,...,km208_pos_05,km208_raw_05,km254_gain_06,km254_pos_06,km254_raw_06,km294_gain_07,km294_pos_07,km294_raw_07,ass_gain_08,ass_raw_08
Bib,,,,,,,,,,,,,,,,,,,,,
319,1.0,False,V. ZALA S. JURGELENAS AGRORODEO,MINI,10:00:00,00:02:18,00:04:01,00:03:38,00:04:43,00:01:11,...,3.0,00:04:43,▲,2.0,00:01:11,▲,1.0,00:00:00,=,00:00:00
302,2.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,09:26:00,00:01:30,00:01:53,00:01:10,00:06:16,00:02:33,...,4.0,00:06:16,▲,3.0,00:02:33,▲,2.0,00:02:04,=,00:02:14
305,3.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,09:35:00,00:00:43,00:01:12,00:10:56,00:10:01,00:05:39,...,9.0,00:10:01,▲,6.0,00:05:39,▲,3.0,00:03:41,=,00:02:50


In [490]:
rb3cTop10 = pd.merge(rb2cTop10[[]], rb3c, how='left', left_index=True,right_index=True)

cols = [c for c in rb3cTop10.columns if c.startswith(('0','1'))]
#rb2cTop10[cols]  = rb2cTop10[cols].apply(lambda x: x.dt.total_seconds())

rb3cTop10['test2']= rb3cTop10[[c for c in rb3cTop10.columns if ('_pos' in c and 'dss' not in c)]+['Pos']].values.tolist()
#Want better rank higher up
rb3cTop10['test2'] = rb3cTop10['test2'].apply(lambda x: [-y if not pd.isnull(y) else float('NaN') for y in x ])
rb3cTop10['Waypoint Rank'] = rb3cTop10['test2'].apply(sparklineStep,figsize=(0.5, 0.5))
rb3cTop10.head(3)

,Pos,Refuel,Crew,Brand,00_dss,01_km47,02_km86,04_km158,05_km208,06_km254,...,km254_gain_06,km254_pos_06,km254_raw_06,km294_gain_07,km294_pos_07,km294_raw_07,ass_gain_08,ass_raw_08,test2,Waypoint Rank
Bib,,,,,,,,,,,,,,,,,,,,,
319,1.0,False,V. ZALA S. JURGELENAS AGRORODEO,MINI,10:00:00,00:02:18,00:04:01,00:03:38,00:04:43,00:01:11,...,▲,2.0,00:01:11,▲,1.0,00:00:00,=,00:00:00,"[-11.0, -10.0, -5.0, -3.0, -2.0, -1.0, -1.0]","<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
302,2.0,False,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,09:26:00,00:01:30,00:01:53,00:01:10,00:06:16,00:02:33,...,▲,3.0,00:02:33,▲,2.0,00:02:04,=,00:02:14,"[-7.0, -6.0, -2.0, -4.0, -3.0, -2.0, -2.0]","<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
305,3.0,False,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,09:35:00,00:00:43,00:01:12,00:10:56,00:10:01,00:05:39,...,▲,6.0,00:05:39,▲,3.0,00:03:41,=,00:02:50,"[-4.0, -2.0, -15.0, -9.0, -6.0, -3.0, -3.0]","<img src=""data:image/png;base64,iVBORw0KGgoAAA..."


In [491]:
rb3cTop10['test2'].head(3)

Bib
319    [-11.0, -10.0, -5.0, -3.0, -2.0, -1.0, -1.0]
302      [-7.0, -6.0, -2.0, -4.0, -3.0, -2.0, -2.0]
305     [-4.0, -2.0, -15.0, -9.0, -6.0, -3.0, -3.0]
Name: test2, dtype: object

In [492]:
#Use a copy for nw, while testing
rb4cTop10 = rb2cTop10.copy()

In [493]:
#Create line chart for pos between each waypoint
q=f"SELECT Bib, WaypointOrder, WaypointPos FROM waypoints WHERE VehicleType='{VTYPE}' AND Stage={STAGE};"
tmp = pd.read_sql(q, conn).pivot(index='Bib',
                                 columns='WaypointOrder',
                                 values='WaypointPos')
tmp.dropna(thresh=MAXMISSING,axis=1,inplace=True)
tmp['waypos'] = tmp.values.tolist()
tmp['waypos'] = tmp['waypos'].apply(lambda x: [-y if not pd.isnull(y) else float('NaN') for y in x ])
tmp['Waypoint Pos'] = tmp['waypos'].apply(sparklineStep,figsize=(0.5, 0.5))
insertColumn(rb4cTop10, 'Waypoint Pos', tmp['Waypoint Pos'], right_of='Pos')

In [494]:
#Get rid of NA cols - maybe do this on a threshold?
rb3cTop10.dropna(how='all',axis=1,inplace=True)

rb3cTop10['test']= rb3cTop10[[c for c in rb3cTop10.columns if (c.startswith(('0','1')) and 'dss' not in c)]].values.tolist()
rb3cTop10['test'] = rb3cTop10['test'].apply(lambda x: [-y if not pd.isnull(y) else float('NaN') for y in x ])
rb3cTop10['Gap to Leader'] =  rb3cTop10['test'].apply(sparkline2, 
                                                      figsize=(0.5, 0.5), 
                                                      dot=True,
                                                      colband=(('pink','lightgreen'),('r.','g.')))

#rb3cTop10.drop('test', axis=1, inplace=True)
#rb3cTop10.drop('test2', axis=1, inplace=True)

insertColumn(rb4cTop10, 'Gap to Leader', rb3cTop10['Gap to Leader'], right_of='Pos')
insertColumn(rb4cTop10, 'Waypoint Rank', rb3cTop10['Waypoint Rank'], right_of='Brand')
rb4cTop10.head(3)

,Road Position,Crew,Brand,Waypoint Rank,Stage Gap,D01_km47,D02_km86,D04_km158,D05_km208,D06_km254,...,Pos,Gap to Leader,Waypoint Pos,01_km47,02_km86,04_km158,05_km208,06_km254,07_km294,Stage Overall
Bib,,,,,,,,,,,,,,,,,,,,,
319,19.0,V. ZALA S. JURGELENAS AGRORODEO,MINI,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",61.0,-42.0,165.0,399.0,NaN,...,1.0,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",61.0,19.0,165.0,564.0,564.0,899.0,927.0
302,3.0,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",109.0,38.0,4783.0,158.0,11.0,...,2.0,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",109.0,147.0,313.0,471.0,482.0,775.0,793.0
305,6.0,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",156.0,32.0,4145.0,519.0,50.0,...,3.0,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",156.0,188.0,-273.0,246.0,296.0,678.0,757.0


In [495]:
#Overall Position, Previous
tmp = get_ranking_data(STAGE, VTYPE,timerank='general')[RANK].head()
tmp.head(3)

,Pos,Bib,Crew,Brand,Time,Gap,Penalty
0,1,319,V. ZALA S. JURGELENAS AGRORODEO,MINI,03:19:04,00:00:00,0 days
1,2,302,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,03:21:18,00:02:14,0 days
2,3,305,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,03:21:54,00:02:50,0 days


In [496]:
rb4cTop10 = pd.merge(rb4cTop10,
                     getCurrPrevOverallRank(STAGE, VTYPE, rebase=REBASER)[['Overall Position',
                                                                            'Previous Overall Position',
                                                                           'Overall Gap', 'Previous']],
                     left_index=True, right_index=True)
moveColumn(rb4cTop10, 'Previous', left_of='Crew')
moveColumn(rb4cTop10, 'Previous Overall Position', left_of='Previous')
moveColumn(rb4cTop10, 'Overall Position', right_of='Pos')
moveColumn(rb4cTop10, 'Overall Gap', left_of='Pos')

moveColumn(rb4cTop10, 'Waypoint Rank', right_of='Gap to Leader')
moveColumn(rb4cTop10, 'Waypoint Pos', left_of='Stage Gap')

rb4cTop10.head(3)

,Road Position,Previous Overall Position,Previous,Crew,Brand,Waypoint Pos,Stage Gap,D01_km47,D02_km86,D04_km158,...,Overall Position,Gap to Leader,Waypoint Rank,01_km47,02_km86,04_km158,05_km208,06_km254,07_km294,Stage Overall
Bib,,,,,,,,,,,,,,,,,,,,,
319,19.0,NaN,NaN,V. ZALA S. JURGELENAS AGRORODEO,MINI,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",61.0,-42.0,165.0,...,1,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",61.0,19.0,165.0,564.0,564.0,899.0,927.0
302,3.0,NaN,NaN,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",109.0,38.0,4783.0,...,2,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",109.0,147.0,313.0,471.0,482.0,775.0,793.0
305,6.0,NaN,NaN,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",156.0,32.0,4145.0,...,3,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...","<img src=""data:image/png;base64,iVBORw0KGgoAAA...",156.0,188.0,-273.0,246.0,296.0,678.0,757.0


In [497]:
if STAGE==1:
    rb4cTop10.drop(['Previous Overall Position','Previous'], axis=1, inplace=True)

In [498]:
s2 = moreStyleDriverSplitReportBaseDataframe(rb4cTop10, STAGE)

display(HTML(s2))

,Road Position,Crew,Brand,Waypoint Pos,Stage Gap,D01_km47,D02_km86,D04_km158,D05_km208,D06_km254,D07_km294,D08_ass,Overall Gap,Pos,Overall Position,Gap to Leader,Waypoint Rank,01_km47,02_km86,04_km158,05_km208,06_km254,07_km294,Stage Overall
Bib,,,,,,,,,,,,,,,,,,,,,,,,
319,19,V. ZALA S. JURGELENAS AGRORODEO,MINI,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACQAAAAkCAYAAADhAJiYAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1%2B/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4xLjEsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy8QZhcZAAABeElEQVRYhe2YwUoCQRjH/8YSSDh7WDdZve1BY7147QW6haeeoceInqI3CIIgpJuXzhGEBAkama6KqNnBw7qtutshGmg1ZgdH2sP8bt/MzsyPb7/ZDzYRBEGAGLHz3wJhpBALZdsHXN7bqNT6a%2BesLMHZcZEtNBwOhQldP7yhOZ4hrydX5hzHWRkTnqGbpzGqjQmNf2QuTg4irRdeQ9XGBM3xjMZ5PYmjghZ5/VZqiCcjYTYSCr8eAH/Wy0ZCmUwm0uK7Vgsv7y4sg9CxYnYX5VIu8h6RhHiwDIKr08NNt6HE7sMohVhw1VC4DdQH018FLQKuDFVqfdQHUxpbBkG5lBMqxH3LRN%2BqMLGrISnEQgqxkEIspBALKcRibS87v33GY2sERVGgKN%2BPuK6L149PFLMqfN9Hu92GpmlQVRXL5RKdTofGi8UCtm0jnU6DEIL5fI5utwtd15FKpeB5Hnq9HkzTXDmbK0OF/T3h3T1MQv6OYSCFWMRO6Av%2Ba3MIi2clPAAAAABJRU5ErkJggg%3D%3D"" />","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJAAAAAkCAYAAABmHbPbAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1%2B/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4xLjEsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy8QZhcZAAAA20lEQVR4nO3ZsQ2CQACGUc7YOIIbsAVbMQRbsQUbOIIllhYQE/1BBN%2BriIpQfLk7uDKO41jBh05b3wD7JiAiAiJyXvXfS5l8dGvb6l7X879vmudx3793LedOXIahunbd9IsFl71l1UX0TEAvLXUryXX/7dzQuiOQB7zDswYiIiAi605hWzF1fo0RiIiAiAiIiICICIiIgIgIiIiAiAiIiICIHHMrI5Fsg/zhFooRiIiAiAiIiICICIiIgIgIiIiAiHiR%2BCt2%2BhJSQEewYXymMCICIiIgIgIiIiAiDyyvND8trSUsAAAAAElFTkSuQmCC"" />",61,-42,165,399,0,335,28,927,1,1,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACQAAAAkCAYAAADhAJiYAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1%2B/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4xLjEsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy8QZhcZAAAB%2BElEQVRYhe2Yy27TQBSGvxkndi6OXaeJhKgAdxFg025Rd5H6Hn0RHqDvwXtUisTCK1YsWCChSkFiwwaBgmRwhsXQQFqr9kxm1C76L61zPJ/OnMvMCKWU4h5J3jXAdT0ANenOgIplwfnbc4plsfW9U2tdVfDlK7jO914E04xiWXD65pSyKgmDkNXrVQPQjxV8%2Bgw4BFJAlsA0Y3G5oKxKKlVRVuWWWT0QgBRQOQQKJLzIAZjnc8Ig3ESoHZBLSQmzZxDpxU%2BenHBxdsHicsE8n2%2BZitrG%2BO07vP8I1Xp3GCH0Vh3NWpn7r7JAwsvD1uZ%2BgeRfmG77zPALlMawv2fk4g9ICpiYwYBPICEgiY3d/AEpBcO%2BsZs/oHigo2Qof0BZYuVWD9SLdhusQQDpyCFQFMLYvEI2Wq9hNHQIBJA/1qVroyiETuAYaNi3KlsA9uy2C5qS%2BvBAt38TBRIyX0BJDIOe2R%2BVso9sIxCYR0nKzbnHD1CWQNi9uWi3U5/0o6FVQ2wPJISOkhAaIB3B8XN4dVwDKmCcWsNA2yPsNIPVT5hkeiRc6WgG7z7ovnMFv0P%2BQNvRIQTkB9swAIM%2BzJ7%2By7G1gth8oJoD3aZHE9hPQaAr0rRNOAcCfb0JQxjbDdT/VX/rsNGv3zqpA7uR4R7IkR5eP5p074D%2BAFMVYdSCtskuAAAAAElFTkSuQmCC"" />","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACQAAAAkCAYAAADhAJiYAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1%2B/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4xLjEsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy8QZhcZAAABlUlEQVRYhe2YTUsCQRjH/0YEEs4e1k3UmweN9eK1L9AtPPUZ%2BhjRp%2BgbBEEQ0iEwqHN0kaAFjTZ3fUPUCA%2B6vm6nBhxfZhd3cQ77uz0zzMyPZ56dGTZk27YNgdjbtQBLIMRj3%2B8Fbl5NFEqNlX1qguDyLMsXarfbngndvX2j0hkirYSX%2BgaDwVKb5xm6f%2B%2BgWO7R%2BF/m%2BvzY0XjPa6hY7qHSGdI4rYRxmpEdj/elhtxkhGUrIXZ7AKytl62EYrGYo8Evuo7PrgU1TmhbNnGAfC7peA5HQm5Q4wS3FyfbTkMR7mAUTsjVlrGnrtbqL9SPF7jKUKHUgNbq01iNE%2BRzSU%2BFXBe110XMIlwNBUI8AiEegRCPQIiHcEJrrw5d1/FsjPBU%2BQUAWJaFr58RsgkJ8/kc1WoVsixDkiTMZjMYhkHj6XQK0zQRjUZBCMFkMkGtVoOiKIhEIhiPx6jX60ilUkvrbszQo9ZduEwzR4eeX6YsoVU/G64ePqA1%2B/R54edlyrIxQ348L3iszNAuEe4rC4R4CCf0B9XViRYk%2BH2nAAAAAElFTkSuQmCC"" />",61,19,165,564,564,899,927
302,3,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACQAAAAkCAYAAADhAJiYAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1%2B/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4xLjEsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy8QZhcZAAABjklEQVRYhe2XvUrDUBSAP0WFOnSRUGrBzSoVxK34Aq7iIIW%2BgPgYjj6B%2BAKCOEhx6%2BIobiJYbIUzCFpKlbbpUOlfHKSFpElv0kiaId92Tjg3H%2Bfce0MWDMMwCBGL8xawEgmpCJ3Qkl2yVqsFJpBIJEyxrZATt891iuVvU%2B5ga42jXc1TzYhNbZXznFnI08iK5W8q9c44rtQ7ji9zqlHhqUMAaS3

In [499]:
if not CHART_POSITIONS:
    TYP='All'
else:
    TYP=f'top_{CHART_POSITIONS}'

img = getTablePNG(s2, fnstub=f'SS{STAGE}_{VTYPE}_{REBASER}_{TYP}')
img

'testpng/SS1_car_310_top_15.png'

In [500]:
from IPython.display import Image
Image(img)